## Sklearn.grid_search

In [1]:
import warnings
warnings.filterwarnings('ignore')

from sklearn import model_selection, datasets, linear_model, metrics

import numpy as np
import pandas as pd

### Генерация датасета

In [2]:
iris = datasets.load_iris()
train_data, test_data, train_label, test_label = model_selection.train_test_split(iris.data, iris.target,
                                                                                  test_size=0.3, random_state=0)

### Задание модели

In [3]:
classifier = linear_model.SGDClassifier(random_state=0, tol=1e-3)

# Генерация сетки
print(*classifier.get_params().items(), sep='\n')

('alpha', 0.0001)
('average', False)
('class_weight', None)
('early_stopping', False)
('epsilon', 0.1)
('eta0', 0.0)
('fit_intercept', True)
('l1_ratio', 0.15)
('learning_rate', 'optimal')
('loss', 'hinge')
('max_iter', 1000)
('n_iter_no_change', 5)
('n_jobs', None)
('penalty', 'l2')
('power_t', 0.5)
('random_state', 0)
('shuffle', True)
('tol', 0.001)
('validation_fraction', 0.1)
('verbose', 0)
('warm_start', False)


### Генерация сетки

In [4]:
parameter_grid = {
    'loss': ['hinge', 'log', 'squared_hinge', 'squared_loss'],
    'penalty': ['l1', 'l2'],
    'max_iter': np.arange(5, 10),
    'alpha': np.linspace(0.0001, 0.001, num=5)
}

cv = model_selection.StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

### Подбор параметров и оценка качества

##### Grid search

In [5]:
grid_cv = model_selection.GridSearchCV(classifier, parameter_grid, scoring='accuracy', cv=cv)

In [6]:
%%time
grid_cv.fit(train_data, train_label)

Wall time: 6.06 s


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
             estimator=SGDClassifier(random_state=0),
             param_grid={'alpha': array([0.0001  , 0.000325, 0.00055 , 0.000775, 0.001   ]),
                         'loss': ['hinge', 'log', 'squared_hinge',
                                  'squared_loss'],
                         'max_iter': array([5, 6, 7, 8, 9]),
                         'penalty': ['l1', 'l2']},
             scoring='accuracy')

In [20]:
print(grid_cv.best_estimator_)

print(grid_cv.best_score_)
print(grid_cv.best_params_)

print()
print(*(f'{k}: {val[:3]}' for k, val in grid_cv.cv_results_.items()), sep='\n')

SGDClassifier(alpha=0.0007750000000000001, max_iter=9, penalty='l1',
              random_state=0)
0.9047619047619048
{'alpha': 0.0007750000000000001, 'loss': 'hinge', 'max_iter': 9, 'penalty': 'l1'}

mean_fit_time: [0.00209348 0.00189419 0.00169585]
std_fit_time: [0.00030552 0.00029881 0.00045663]
mean_score_time: [3.04102898e-04 9.97304916e-05 2.98833847e-04]
std_score_time: [0.00046453 0.00029919 0.00045648]
param_alpha: [0.0001 0.0001 0.0001]
param_loss: ['hinge' 'hinge' 'hinge']
param_max_iter: [5 5 6]
param_penalty: ['l1' 'l2' 'l1']
params: [{'alpha': 0.0001, 'loss': 'hinge', 'max_iter': 5, 'penalty': 'l1'}, {'alpha': 0.0001, 'loss': 'hinge', 'max_iter': 5, 'penalty': 'l2'}, {'alpha': 0.0001, 'loss': 'hinge', 'max_iter': 6, 'penalty': 'l1'}]
split0_test_score: [0.76190476 0.71428571 0.71428571]
split1_test_score: [0.80952381 0.61904762 0.76190476]
split2_test_score: [0.52380952 0.38095238 0.71428571]
split3_test_score: [0.61904762 0.61904762 0.71428571]
split4_test_score: [0.4761

#### Randomized grid search

In [21]:
randomized_grid_cv = model_selection.RandomizedSearchCV(classifier, parameter_grid,
                                                        scoring='accuracy', cv=cv,
                                                        n_iter=20, random_state=0)

In [22]:
%%time
randomized_grid_cv.fit(train_data, train_label)

Wall time: 748 ms


RandomizedSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
                   estimator=SGDClassifier(random_state=0), n_iter=20,
                   param_distributions={'alpha': array([0.0001  , 0.000325, 0.00055 , 0.000775, 0.001   ]),
                                        'loss': ['hinge', 'log',
                                                 'squared_hinge',
                                                 'squared_loss'],
                                        'max_iter': array([5, 6, 7, 8, 9]),
                                        'penalty': ['l1', 'l2']},
                   random_state=0, scoring='accuracy')

In [23]:
print(randomized_grid_cv.best_estimator_)

print(randomized_grid_cv.best_score_)
print(randomized_grid_cv.best_params_)

print()
print(*(f'{k}: {val[:3]}' for k, val in randomized_grid_cv.cv_results_.items()), sep='\n')

SGDClassifier(alpha=0.0007750000000000001, loss='squared_hinge', max_iter=8,
              penalty='l1', random_state=0)
0.8666666666666666
{'penalty': 'l1', 'max_iter': 8, 'loss': 'squared_hinge', 'alpha': 0.0007750000000000001}

mean_fit_time: [0.00289078 0.00379136 0.00239418]
std_fit_time: [0.0008249  0.00146638 0.00048927]
mean_score_time: [0.0003011  0.00069668 0.00019941]
std_score_time: [0.00045997 0.0004561  0.00039883]
param_penalty: ['l1' 'l1' 'l2']
param_max_iter: [9 5 8]
param_loss: ['log' 'log' 'squared_hinge']
param_alpha: [0.0001 0.001 0.00055]
params: [{'penalty': 'l1', 'max_iter': 9, 'loss': 'log', 'alpha': 0.0001}, {'penalty': 'l1', 'max_iter': 5, 'loss': 'log', 'alpha': 0.001}, {'penalty': 'l2', 'max_iter': 8, 'loss': 'squared_hinge', 'alpha': 0.00055}]
split0_test_score: [0.71428571 0.95238095 0.9047619 ]
split1_test_score: [0.71428571 0.85714286 0.85714286]
split2_test_score: [0.71428571 0.66666667 0.95238095]
split3_test_score: [0.71428571 0.71428571 0.9047619 ]
